In [1]:
# Importação de bibliotecas
import pandas as pd

In [7]:
# Variáveis utilizadas
fonte = 'is_agro'
dataset_name = 'usodaterra_agro'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Prata

In [8]:
df = (
    pd.read_excel(f'{raiz}/bronze_{fonte}_{dataset_name}_br.xlsx', header=1) # Arquivo gerado pela equipe da Embrapa de Campinas
    .melt(
        id_vars=['cod_mapbiomas', 'classes'],
        var_name='data',
        value_name='valor'
    )
    .assign(data = lambda dt: pd.to_datetime(dt.data, format='%Y'))
    .convert_dtypes()
)

df.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_br.parquet') # Exportação para a camada prata

In [4]:
df_lavtemp = (
    pd.read_csv(f'{raiz}/ouro_volume_agricola_lavouras_temporarias_uf.csv')
    .groupby(['geocodigo', 'data'])
    .agg({'valor': 'sum'})
    .reset_index()
    .assign(data = lambda dt: pd.to_datetime(dt.data, format='ISO8601'))
    .convert_dtypes()
)
df_lavtemp_br = (
    df_lavtemp
    .groupby('data')
    .agg({'valor': 'sum'})
    .reset_index()
    .assign(data = lambda dt: pd.to_datetime(dt.data, format='ISO8601'))
    .convert_dtypes()
)

df_lavperene = (
    pd.read_csv(f'{raiz}/ouro_volume_agricola_lavouras_permanentes_uf.csv')
    .assign(data = lambda dt: pd.to_datetime(dt.data, format='ISO8601'))
    .convert_dtypes()
)
df_lavperene_br = (
    df_lavperene
    .groupby('data')
    .agg({'area': 'sum'})
    .reset_index()
    .assign(data = lambda dt: pd.to_datetime(dt.data, format='ISO8601'))
    .convert_dtypes()
)

In [5]:
df_is_agro_lavtemp = (
    df_lavtemp
    .merge(
        df_lavtemp_br,
        how='left',
        on='data',
        suffixes=['_uf', '_br']
    )
    .merge(
        df
        .query('cod_mapbiomas == 19'),
        how='left',
        on='data'
    )
    .assign(value = lambda n: n.valor_uf * n.valor/n.valor_br)
    .filter(['geocodigo', 'data', 'value'])
    .convert_dtypes()
)

In [6]:
df_is_agro_lavperene = (
    df_lavperene
    .merge(
        df_lavperene_br,
        how='left',
        on='data',
        suffixes=['_uf', '_br']
    )
    .merge(
        df
        .query('cod_mapbiomas == 36'),
        how='left',
        on='data'
    )
    .assign(value = lambda n: n.area_uf * n.valor/n.area_br)
    .filter(['geocodigo', 'data', 'value'])
    .convert_dtypes()
)